In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from imblearn.over_sampling import SMOTE
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.feature_selection import VarianceThreshold
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

https://github.com/Ilincalink/ML-fundamentals-2025.git

In [2]:
df= pd.read_csv("hour.csv")

In [ ]:
df.head

# Step 1

In [ ]:
# Plot the distribution of the target variable
plt.figure(figsize=(10, 6))
sns.histplot(df['cnt'], kde=True, bins=30, color='blue')
plt.title('Distribution of variable (cnt)')
plt.xlabel('cnt')
plt.ylabel('Frequency')
plt.show()

# Check skewness of the target variable
print(f'Skewness of cnt: {df["cnt"].skew()}')

from this first graph we can see that the distribution is very skewed to the right, meaning that the mode is to the furthest left, followed by the median and then the mean. The skeweness value of 1.2774 further proves that this is a positive skew, with most values being to the left and the aforementioned positions of the median mean and mode. The right hand-side also shows there might be some outliers I have to take into consideration, that could otherwise distrupt the future system.

In [ ]:
# Plot cnt by hour with the updated method
plt.figure(figsize=(10, 6))
sns.boxplot(x='hr', y='cnt', data=df, hue='hr', legend=False)
plt.title('cnt by Hour')
plt.xlabel('Hour of the Day')
plt.ylabel('cnt')
plt.show()


We can see that the count changes depending on the hour, with the peaks at 8 o'clock and 17 o'clock both with approximately 600. 

In [ ]:
# Plot cnt by weekday with the updated method
plt.figure(figsize=(10, 6))
sns.boxplot(x='weekday', y='cnt', data=df, hue='weekday', legend=False)
plt.title('cnt by Weekday')
plt.xlabel('Weekday')
plt.ylabel('cnt')
plt.show()


The weekday plot shows a more distributed pattern, showing that the count is balanced throughout the week. 

In [ ]:
# Plot cnt by month with the updated method
plt.figure(figsize=(10, 6))
sns.boxplot(x='mnth', y='cnt', data=df, hue='mnth', legend=False)
plt.title('cnt by Month')
plt.xlabel('Month')
plt.ylabel('cnt')
plt.show()


The plot above shows how the usage of bicycles grows in the hot months and gradually lowers as the weather gets colder. 

In [ ]:
# Plot cnt by season with the updated method
plt.figure(figsize=(10, 6))
sns.boxplot(x='season', y='cnt', data=df, hue='season', legend=False)
plt.title('cnt by Season')
plt.xlabel('Season')
plt.ylabel('cnt')
plt.show()


As fully expected, the highest usage of the bikes is in the summer season, followed by spring and autumn. 

In [ ]:
# Plot cnt by holiday with the updated method
plt.figure(figsize=(10, 6))
sns.boxplot(x='holiday', y='cnt', data=df, hue='holiday', legend=False)
plt.title('cnt by Holiday')
plt.xlabel('Holiday')
plt.ylabel('cnt')
plt.show()



In [ ]:
# Plot cnt by workingday with the updated method
plt.figure(figsize=(10, 6))
sns.boxplot(x='workingday', y='cnt', data=df, hue='workingday', legend=False)
plt.title('cnt by Working Day')
plt.xlabel('Working Day')
plt.ylabel('cnt')
plt.show()


In [ ]:
# Plot cnt by temperature (temp)
plt.figure(figsize=(10, 6))
sns.scatterplot(x='temp', y='cnt', data=df, color='green')
plt.title('cnt by Temperature')
plt.xlabel('Temperature')
plt.ylabel('cnt')
plt.show()

# Plot cnt by apparent temperature (atemp)
plt.figure(figsize=(10, 6))
sns.scatterplot(x='atemp', y='cnt', data=df, color='red')
plt.title('cnt by Apparent Temperature (atemp)')
plt.xlabel('Apparent Temperature')
plt.ylabel('cnt')
plt.show()

# Plot cnt by humidity (hum)
plt.figure(figsize=(10, 6))
sns.scatterplot(x='hum', y='cnt', data=df, color='blue')
plt.title('cnt by Humidity')
plt.xlabel('Humidity')
plt.ylabel('cnt')
plt.show()

# Plot cnt by windspeed
plt.figure(figsize=(10, 6))
sns.scatterplot(x='windspeed', y='cnt', data=df, color='purple')
plt.title('cnt by Windspeed')
plt.xlabel('Windspeed')
plt.ylabel('cnt')
plt.show()


I have decided to gather the plots regarding humidity, temperature and windspeed together as I believe having them one after another can give more insight into how such weather events truly effect our 'cnt' variable. From the plots above we can see a strong correlation between weather and the cnt of bikes. The higher the temperature, the higher the usage of bikes, whereas higher wind speeds lower the number of people using this method of transport. I found the humidity plot to be the most interesting one, as we can see an almost normal distribution between 'cnt' and this change in weather. 

In [ ]:
# Plot cnt by weather situation (weathersit)
plt.figure(figsize=(10, 6))
sns.boxplot(x='weathersit', y='cnt', data=df, hue='weathersit', legend=False)
plt.title('cnt by Weather Situation')
plt.xlabel('Weather Situation')
plt.ylabel('cnt')
plt.show()


In [ ]:
# Check for outliers using boxplots
plt.figure(figsize=(10, 6))
sns.boxplot(x='cnt', data=df, color='orange')
plt.title('Outliers in cnt')
plt.show()


In [ ]:
# Drop the irrelevant columns
df = df.drop(columns=['instant', 'dteday', 'casual', 'registered'])

# Verify the new structure of the dataframe
df.head()


In [ ]:
correlation_matrix = df.corr()
plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, 
            annot=True, 
            cmap='coolwarm', 
            center=0,
            fmt='.2f',
            square=True)
plt.title('Correlation Matrix Heatmap')
plt.tight_layout()
plt.show()


From this correlation table we can see that atemp and temp are very highly correlated and that this could also be the case for holiday and workingday even though it is not as drastic. I will look into this in the future to make sure no other variables are 'leaky' or redundant. 

# Step 2

In [ ]:
df = df.sort_values(by=['yr', 'mnth', 'hr'])

# Calculate the number of rows in the dataset
n_rows = len(df)

# Calculate split indices
train_size = int(0.6 * n_rows)
validation_size = int(0.2 * n_rows)

# Split the dataset
train_data = df[:train_size]
validation_data = df[train_size:train_size + validation_size]
test_data = df[train_size + validation_size:]

train_data['cnt'] = train_data['cnt'].astype(float)
validation_data['cnt'] = validation_data['cnt'].astype(float)
test_data['cnt'] = test_data['cnt'].astype(float)

# Now apply log transformation
train_data['cnt'] = np.log1p(train_data['cnt'])
validation_data['cnt'] = np.log1p(validation_data['cnt'])
test_data['cnt'] = np.log1p(test_data['cnt'])


# Print the size of each split
print(f"Training set size: {train_data.shape[0]}")
print(f"Validation set size: {validation_data.shape[0]}")
print(f"Test set size: {test_data.shape[0]}")


In [ ]:
# Check the total number of rows in the dataset
total_rows = len(df)
print(f"Total number of rows in the dataset: {total_rows}")

# Verify the sum of the splits
assert (train_data.shape[0] + validation_data.shape[0] + test_data.shape[0]) == total_rows, "The split sizes don't add up!"


To make sure my data in temporaly correct and put in order, I ordered it by month, year, hour, to make sure that the training set was before the validation which came before the test set. I did this to ensure that my groups would not contain data leakage therefore giving me better results. Finally, in the second part of my code I checked the total number of rows in my dataset to make sure that it is the sum that i get from adding the test, training and validation sets in order to make sure all my data is used efficiently and effectively. 

# Step 3

In [ ]:

# Step 1: Encode cyclical features (hr, weekday)
def encode_cyclical(df, column, max_value):
    df[column + '_sin'] = np.sin(2 * np.pi * df[column] / max_value)
    df[column + '_cos'] = np.cos(2 * np.pi * df[column] / max_value)
    return df

# Apply cyclical encoding to 'hr' (hour of the day) and 'weekday'
df = encode_cyclical(df, 'hr', 24)  # 24 hours in a day
df = encode_cyclical(df, 'weekday', 7)  # 7 days in a week

# Step 2: One-hot encoding for categorical features (season, weathersit, mnth)
# We can apply this after splitting to avoid data leakage
categorical_columns = ['season', 'weathersit', 'mnth']

# Step 3: Apply scaling (StandardScaler) to continuous features (temp, hum, windspeed)
# Remove 'atemp' from the continuous features list as it's dropped later in the code
continuous_columns = ['temp', 'hum', 'windspeed', 'temp_hum_interaction']

# Step 4: Interaction terms (temp * hum)
df['temp_hum_interaction'] = df['temp'] * df['hum']

# Step 5: Remove leaky or redundant features (e.g., drop 'atemp' if highly correlated with 'temp')
df.drop(columns=['atemp'], inplace=True)  # Dropping 'atemp' column

# Splitting the data into training, validation, and test sets
# Assuming you have already split the dataset as per the previous task
train_data = df[:10427]
validation_data = df[10427:10427+3475]
test_data = df[10427+3475:]

# Separate target variable 'cnt' from features
X_train = train_data.drop(columns=['cnt'])
y_train = train_data['cnt']
X_val = validation_data.drop(columns=['cnt'])
y_val = validation_data['cnt']
X_test = test_data.drop(columns=['cnt'])
y_test = test_data['cnt']

# Column Transformer: Apply one-hot encoding and scaling
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(drop='first'), categorical_columns),  # One-hot encode categorical columns
        ('num', StandardScaler(), continuous_columns),  # Scale continuous features and interaction terms
        ('cyc', 'passthrough', ['hr_sin', 'hr_cos', 'weekday_sin', 'weekday_cos'])  # Keep cyclical features as they are
    ]
)

# Create a pipeline for preprocessing
pipeline = Pipeline(steps=[('preprocessor', preprocessor)])

# Fit the preprocessor on the training data and apply to the validation and test sets
X_train_transformed = pipeline.fit_transform(X_train)
X_val_transformed = pipeline.transform(X_val)
X_test_transformed = pipeline.transform(X_test)

# Verify the transformed shapes
print(f"Transformed X_train shape: {X_train_transformed.shape}")
print(f"Transformed X_val shape: {X_val_transformed.shape}")
print(f"Transformed X_test shape: {X_test_transformed.shape}")


# Step 4

## Linear regression model

In [ ]:
print(df.columns)


In [ ]:

model = LinearRegression()
model.fit(X_train_transformed, y_train)

# Predict on the validation set
y_val_pred = model.predict(X_val_transformed)

# Calculate performance metrics
mse = mean_squared_error(y_val, y_val_pred)
mae = mean_absolute_error(y_val, y_val_pred)
r2 = r2_score(y_val, y_val_pred)

# Print the evaluation metrics
print(f"Mean Squared Error (MSE): {mse}")
print(f"Mean Absolute Error (MAE): {mae}")
print(f"R² Score: {r2}")


In [ ]:
# Calculate residuals
residuals = y_val - y_val_pred

# Plotting residuals
plt.figure(figsize=(10,6))
sns.histplot(residuals, kde=True)
plt.title('Residuals Distribution')
plt.xlabel('Residuals')
plt.ylabel('Frequency')
plt.show()

# Plotting residuals vs predicted values
plt.figure(figsize=(10,6))
plt.scatter(y_val_pred, residuals)
plt.axhline(y=0, color='r', linestyle='--')
plt.title('Residuals vs Predicted')
plt.xlabel('Predicted values')
plt.ylabel('Residuals')
plt.show()


## Step 5


In [ ]:
from sklearn.ensemble import RandomForestRegressor

# Initialize Random Forest Regressor with default parameters
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

# Train the model on the transformed training data
rf_model.fit(X_train_transformed, y_train)


In [ ]:
# Predict on validation set
y_val_rf_pred = rf_model.predict(X_val_transformed)

# Evaluate performance
rf_mse = mean_squared_error(y_val, y_val_rf_pred)
rf_mae = mean_absolute_error(y_val, y_val_rf_pred)
rf_r2 = r2_score(y_val, y_val_rf_pred)

# Print the metrics
print("Random Forest Regressor Performance:")
print(f"Mean Squared Error (MSE): {rf_mse}")
print(f"Mean Absolute Error (MAE): {rf_mae}")
print(f"R² Score: {rf_r2}")


In [ ]:
print("\n--- Comparison with Linear Regression ---")
print(f"Linear Regression R²: {r2:.4f}")
print(f"Random Forest R²: {rf_r2:.4f}")


In [ ]:
importances = rf_model.feature_importances_

# Get feature names from the pipeline
# You have one-hot encoded features, so we’ll extract all feature names
ohe = pipeline.named_steps['preprocessor'].named_transformers_['cat']
ohe_feature_names = ohe.get_feature_names_out(categorical_columns)
all_feature_names = list(ohe_feature_names) + continuous_columns + ['hr_sin', 'hr_cos', 'weekday_sin', 'weekday_cos']

# Create a DataFrame for visualization
feature_importances_df = pd.DataFrame({
    'Feature': all_feature_names,
    'Importance': importances
}).sort_values(by='Importance', ascending=False)

# Plot
plt.figure(figsize=(12, 8))
sns.barplot(data=feature_importances_df.head(15), x='Importance', y='Feature')
plt.title('Top 15 Feature Importances - Random Forest')
plt.tight_layout()
plt.show()


## Step 6


In [ ]:
from lightgbm import LGBMRegressor

lgb_model = LGBMRegressor(n_estimators=100, learning_rate=0.1, random_state=42)
lgb_model.fit(X_train_transformed, y_train)
y_val_lgb_pred = lgb_model.predict(X_val_transformed)


In [ ]:
# Evaluate on the validation set
lgb_mse = mean_squared_error(y_val, y_val_lgb_pred)
lgb_mae = mean_absolute_error(y_val, y_val_lgb_pred)
lgb_r2 = r2_score(y_val, y_val_lgb_pred)

# Print the metrics
print("LightGBM Regressor Performance:")
print(f"Mean Squared Error (MSE): {lgb_mse}")
print(f"Mean Absolute Error (MAE): {lgb_mae}")
print(f"R² Score: {lgb_r2}")


In [ ]:
# Calculate residuals
residuals_lgb = y_val - y_val_lgb_pred

# Residuals Distribution
plt.figure(figsize=(10, 6))
sns.histplot(residuals_lgb, kde=True)
plt.title('LightGBM Residuals Distribution')
plt.xlabel('Residuals')
plt.ylabel('Frequency')
plt.show()

# Residuals vs Predicted
plt.figure(figsize=(10, 6))
plt.scatter(y_val_lgb_pred, residuals_lgb, alpha=0.5)
plt.axhline(y=0, color='r', linestyle='--')
plt.title('LightGBM Residuals vs Predicted')
plt.xlabel('Predicted values')
plt.ylabel('Residuals')
plt.show()


In [ ]:
print("\n--- R² Score Comparison ---")
print(f"Linear Regression R²:      {r2:.4f}")
print(f"Random Forest R²:          {rf_r2:.4f}")
print(f"LightGBM Regressor R²:     {lgb_r2:.4f}")


In [ ]:
importances_lgb = lgb_model.feature_importances_

ohe = pipeline.named_steps['preprocessor'].named_transformers_['cat']
ohe_feature_names = ohe.get_feature_names_out(categorical_columns)
all_feature_names = list(ohe_feature_names) + continuous_columns + ['hr_sin', 'hr_cos', 'weekday_sin', 'weekday_cos']

feature_importances_lgb_df = pd.DataFrame({
    'Feature': all_feature_names,
    'Importance': importances_lgb
}).sort_values(by='Importance', ascending=False)

# Plot top 15 important features
plt.figure(figsize=(12, 8))
sns.barplot(data=feature_importances_lgb_df.head(15), x='Importance', y='Feature')
plt.title('Top 15 Feature Importances - LightGBM')
plt.tight_layout()
plt.show()


# overfitting and high variance checks

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

# Calculate MSE and R² for the training data
y_train_lgb_pred = lgb_model.predict(X_train_transformed)
train_mse = mean_squared_error(y_train, y_train_lgb_pred)
train_r2 = r2_score(y_train, y_train_lgb_pred)

# Calculate MSE and R² for the validation data
val_mse = mean_squared_error(y_val, y_val_lgb_pred)
val_r2 = r2_score(y_val, y_val_lgb_pred)

print(f"Training MSE: {train_mse}")
print(f"Training R²: {train_r2}")
print(f"Validation MSE: {val_mse}")
print(f"Validation R²: {val_r2}")



Signs of overfitting:

Training R² is much higher than Validation R².

Training MSE is much lower than Validation MSE.
Training R² is much higher than Validation R²:

Overfitting occurs when the model fits the training data too well, capturing noise or outliers that do not generalize to new, unseen data.

If the R² score for the training set is significantly higher than for the validation set, it indicates that the model is fitting the training data too closely and might not generalize well to new data.

Training MSE is much lower than Validation MSE:

MSE (Mean Squared Error) is a measure of how far off the model’s predictions are from the actual values.

If the MSE for the training set is much lower than for the validation set, it suggests that the model is overfitting: it performs very well on the training set (low MSE) but struggles on unseen data (higher MSE on the validation set).

Signs of High Variance in Your Model:
Let’s focus on the LightGBM model you've been working on:

Training R² vs Validation R²:

High training R² but low validation R² indicates high variance. If your model has a much higher performance on the training set than on the validation set, it's overfitting and has high variance.

Training MSE vs Validation MSE:

Similar to the R² comparison, if training MSE is much lower than validation/test MSE, it’s a sign that the model is fitting to noise in the training data, causing poor performance on unseen data (high variance).



## Step 7

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

# Define the model
rf_model = RandomForestRegressor(random_state=42)

# Hyperparameter grid (wider ranges for exploration)
param_dist = {
    'n_estimators': randint(200, 1000),  # Increased number of trees
    'max_depth': randint(10, 150),        # Larger range for max depth to capture more complexity
    'min_samples_split': randint(2, 20),  # Minimum samples required to split
    'min_samples_leaf': randint(1, 20),   # Minimum samples required at leaf
    'max_features': ['sqrt', 'log2', None]  # Testing different values for max features
}

# RandomizedSearchCV with 5-fold cross-validation
random_search_rf = RandomizedSearchCV(estimator=rf_model, param_distributions=param_dist, 
                                      n_iter=100, cv=5, n_jobs=-1, verbose=2, random_state=42)

# Fit the model
random_search_rf.fit(X_train_transformed, y_train)

# Best parameters and performance
print("Best Parameters for Random Forest:", random_search_rf.best_params_)
print("Best Cross-Validation Score for Random Forest:", random_search_rf.best_score_)

# Get the best model and evaluate performance on validation set
best_rf_model = random_search_rf.best_estimator_
y_val_rf_pred = best_rf_model.predict(X_val_transformed)

# Evaluate performance on validation set
from sklearn.metrics import mean_squared_error, r2_score
print(f"Random Forest MSE on Validation Set: {mean_squared_error(y_val, y_val_rf_pred)}")
print(f"Random Forest R² on Validation Set: {r2_score(y_val, y_val_rf_pred)}")


Cross-Validation Score vs. Validation R²:

The best cross-validation score (0.5155) is relatively close to the R² score on the validation set (0.656), indicating that the model is performing reasonably well.

This suggests that the model generalizes well on unseen data, but there might still be room for improvement.

MSE on Validation:

The MSE of 15933.13 on the validation set is relatively high, which suggests that there’s still a considerable amount of error in the predictions. In real-world cases, you'd want this value to be lower.

R² on Validation:

The R² value of 0.657 indicates that about 65.7% of the variance in the target variable is explained by the model. This is a decent start, but there is still room to improve.



In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, r2_score

# Evaluate the Random Forest model on the validation set
y_val_rf_pred = random_search_rf.best_estimator_.predict(X_val_transformed)

# Calculate MSE and R² on the validation set
mse_val = mean_squared_error(y_val, y_val_rf_pred)
r2_val = r2_score(y_val, y_val_rf_pred)

# Print validation performance
print(f"Random Forest MSE on Validation Set: {mse_val}")
print(f"Random Forest R² on Validation Set: {r2_val}")

# Now, plot the feature importance
importances = random_search_rf.best_estimator_.feature_importances_
indices = importances.argsort()

# Get actual feature names (assuming you're using a ColumnTransformer or OneHotEncoder pipeline)
try:
    # Try to get feature names from pipeline
    feature_names = X_train_transformed.columns
except AttributeError:
    # If it's not a DataFrame, use the transformer pipeline to get names
    try:
        feature_names = preprocessor.get_feature_names_out()
    except:
        feature_names = [f'Feature {i}' for i in range(X_train_transformed.shape[1])]

# Plot top N features
top_n = min(10, X_train_transformed.shape[1])

plt.figure(figsize=(10, 6))
plt.title("Random Forest Feature Importance")
plt.barh(range(top_n), importances[indices[-top_n:]], align="center")
plt.yticks(range(top_n), [feature_names[i] for i in indices[-top_n:]])  # Use actual feature names
plt.xlabel("Feature Importance")
plt.tight_layout()
plt.show()


# Step 8

# Step 9